<a href="https://colab.research.google.com/github/hanene2030/pandas/blob/main/15_different_merging_types.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Merging types


In [1]:
import pandas as pd
students = pd.read_csv("students.csv")
grades = pd.read_csv("grades1.csv")
contacts = pd.read_csv("contact.csv")

In [2]:
grades.tail()

,student_id,course,grade
120,34312,DESN101,E
121,34313,DESN101,D
122,34314,DESN101,A
123,34315,DESN101,A
124,34316,DESN101,A


In [3]:
grades_design = grades[grades.course == "DESN101"]
print(grades_design.shape, students.shape)
# we have 25 students, but only 12 took the design course 

(12, 3) (25, 3)


## Merge types

In [4]:
df = pd.merge(students, grades_design, left_on="id", right_on="student_id")
print(len(df))

12


In [5]:
df = pd.merge(students, grades_design, left_on="id", right_on="student_id", how="inner")
print(len(df))

12


In [6]:
df = pd.merge(students, grades_design, left_on="id", right_on="student_id", how="left")
print(len(df))

25


In [7]:
df.sample(5)

,id,firstname,lastname,student_id,course,grade
14,34306,Mark,Phillips,NaN,NaN,NaN
18,34310,William,Sanders,34310.0,DESN101,C
20,34312,Douglas,Cook,34312.0,DESN101,E
0,34292,Joshua,Davis,NaN,NaN,NaN
23,34315,Russell,Reed,34315.0,DESN101,A


In [8]:
df = pd.merge(students, grades_design, left_on="id", right_on="student_id", how="right")
print(len(df))

12


In [9]:
# outer join
df_a = pd.DataFrame({"A":["x","y","z"], "B":[1,2,3]})
df_b = pd.DataFrame({"A":["u","v","x"], "C":[5.0, 4.0, 3.0]})
display(df_a,df_b)

,A,B
0,x,1
1,y,2
2,z,3


,A,C
0,u,5.0
1,v,4.0
2,x,3.0


In [10]:
pd.merge(df_a, df_b,on="A")

,A,B,C
0,x,1,3.0


In [11]:
pd.merge(df_a, df_b,on="A", how="left")

,A,B,C
0,x,1,3.0
1,y,2,NaN
2,z,3,NaN


In [12]:
pd.merge(df_a, df_b,on="A", how="right")

,A,B,C
0,u,NaN,5.0
1,v,NaN,4.0
2,x,1.0,3.0


In [13]:
pd.merge(df_a, df_b,on="A", how="outer")

,A,B,C
0,x,1.0,3.0
1,y,2.0,NaN
2,z,3.0,NaN
3,u,NaN,5.0
4,v,NaN,4.0


## Duplicate keys

In [14]:
# duplicate keys
df_a = pd.DataFrame({"A":["x","x","z"], "B":[1,2,3]})
df_b = pd.DataFrame({"A":["u","x","x"], "C":[5.0, 4.0, 3.0]})
display(df_a,df_b)

,A,B
0,x,1
1,x,2
2,z,3


,A,C
0,u,5.0
1,x,4.0
2,x,3.0


In [15]:
pd.merge(df_a, df_b,on="A")

,A,B,C
0,x,1,4.0
1,x,1,3.0
2,x,2,4.0
3,x,2,3.0


## Duplicate rows

In [16]:
df_a = pd.DataFrame({"A":["x","y","z"], "B":[1,2,3]})
df_b = pd.DataFrame({"A":["u","v","x"], "B":[5.0, 4.0, 3.0]})
display(df_a,df_b)

,A,B
0,x,1
1,y,2
2,z,3


,A,B
0,u,5.0
1,v,4.0
2,x,3.0


In [17]:
pd.merge(df_a, df_b,on="A")

,A,B_x,B_y
0,x,1,3.0


In [18]:
df = pd.merge(df_a, df_b,on="A", suffixes=("_left","_right"))

In [19]:
df.columns = ["A", "left", "right"]
df

,A,left,right
0,x,1,3.0


In [20]:
df.rename(columns={"left":"B_x", "right" : "B_y"})

,A,B_x,B_y
0,x,1,3.0


## Validating expected outputs

In [21]:
# All students should have a contact
pd.merge(students, contacts, left_on="id", right_on="student_id", validate="one_to_one").head(3)

,id,firstname,lastname,student_id,parent_contact,phone
0,34292,Joshua,Davis,34292,Samuel Davis,(356) 849-0352
1,34293,Karen,Flores,34293,Laura Flores,(477) 325-7117
2,34294,Julia,Walker,34294,Eric Walker,(871) 639-0797


In [22]:
# duplicate keys
df_a = pd.DataFrame({"A":["x","x","z"], "B":[1,2,3]})
df_b = pd.DataFrame({"A":["u","x","x"], "C":[5.0, 4.0, 3.0]})
#pd.merge(df_a, df_b,on="A",validate="one_to_one") ==Eroor


In [23]:
students_2 = students.append(students.iloc[0])
display(students_2.head(2), students_2.tail(2))

,id,firstname,lastname
0,34292,Joshua,Davis
1,34293,Karen,Flores


,id,firstname,lastname
24,34316,Stephen,Martinez
0,34292,Joshua,Davis


In [24]:
#pd.merge(students_2, contacts, left_on="id", right_on="student_id", validate="one_to_one").head(3)
#==>Error

In [25]:
pd.merge(students_2, contacts, left_on="id", right_on="student_id", validate="many_to_one").head(3)

,id,firstname,lastname,student_id,parent_contact,phone
0,34292,Joshua,Davis,34292,Samuel Davis,(356) 849-0352
1,34292,Joshua,Davis,34292,Samuel Davis,(356) 849-0352
2,34293,Karen,Flores,34293,Laura Flores,(477) 325-7117


## Composite keys

In [26]:
df_1 = pd.DataFrame({"year": [2000, 2000, 2001, 2001], "sem": [1, 2, 1, 2], 
                     "fee": [200, 200, 200, 200]})
df_2 = pd.DataFrame({"year": [2000, 2000, 2001, 2001], "sem": [1, 2, 1, 2], 
                     "student": [1, 2, 2, 3], "discount": [0.1, 0.2, 0.2, 1.0]})
df_3 = pd.DataFrame({"student": [1, 2, 3, 4, 5]})

display(df_1, df_2, df_3)
#from IPython.display import Javascript
#Javascript('this.element.attr("style", "flex-direction: row;")')


,year,sem,fee
0,2000,1,200
1,2000,2,200
2,2001,1,200
3,2001,2,200


,year,sem,student,discount
0,2000,1,1,0.1
1,2000,2,2,0.2
2,2001,1,2,0.2
3,2001,2,3,1.0


,student
0,1
1,2
2,3
3,4
4,5


In [27]:
combined = pd.merge(df_1,df_2, on=["year", "sem"])
combined

,year,sem,fee,student,discount
0,2000,1,200,1,0.1
1,2000,2,200,2,0.2
2,2001,1,200,2,0.2
3,2001,2,200,3,1.0


In [28]:
combined["due"] = combined.fee * (1 - combined.discount)
combined

,year,sem,fee,student,discount,due
0,2000,1,200,1,0.1,180.0
1,2000,2,200,2,0.2,160.0
2,2001,1,200,2,0.2,160.0
3,2001,2,200,3,1.0,0.0


In [29]:
pd.merge(df_3, df_2, on="student", how="left")

,student,year,sem,discount
0,1,2000.0,1.0,0.1
1,2,2000.0,2.0,0.2
2,2,2001.0,1.0,0.2
3,3,2001.0,2.0,1.0
4,4,NaN,NaN,NaN
5,5,NaN,NaN,NaN


In [30]:
df_1["key"], df_3["key"] = 1, 1
df_cross = pd.merge(df_1, df_3, on="key").drop("key", axis=1)
df_cross.head()
# every students fees for every semester and year

,year,sem,fee,student
0,2000,1,200,1
1,2000,1,200,2
2,2000,1,200,3
3,2000,1,200,4
4,2000,1,200,5


In [33]:
all_fees = pd.merge(df_2, df_cross, on=["sem","year","student"],how='right')
all_fees.head(3)

,year,sem,student,discount,fee
0,2000,1,1,0.1,200
1,2000,1,2,NaN,200
2,2000,1,3,NaN,200


In [34]:
all_fees.discount.fillna(0, inplace=True)
all_fees["due"] = all_fees.fee * (1 - all_fees.discount) 
all_fees

,year,sem,student,discount,fee,due
0,2000,1,1,0.1,200,180.0
1,2000,1,2,0.0,200,200.0
2,2000,1,3,0.0,200,200.0
3,2000,1,4,0.0,200,200.0
4,2000,1,5,0.0,200,200.0
5,2000,2,1,0.0,200,200.0
6,2000,2,2,0.2,200,160.0
7,2000,2,3,0.0,200,200.0
8,2000,2,4,0.0,200,200.0
9,2000,2,5,0.0,200,200.0
